In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ny-ems-incident-dispatch-data/EMS_incident_dispatch_data_description.xlsx
/kaggle/input/ny-ems-incident-dispatch-data/socrata_metadata.json
/kaggle/input/ny-ems-incident-dispatch-data/ems-incident-dispatch-data.csv


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# df = pd.read_csv('/kaggle/input/ny-ems-incident-dispatch-data/ems-incident-dispatch-data.csv', nrows=500000)
#df = pd.read_csv('/kaggle/input/ny-ems-incident-dispatch-data/ems-incident-dispatch-data.csv')


# Iterates through all columns and modifies the data type to reduce memory usage
def reduce_mem_usage(df):
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


# Creates a dataframe and optimizes its memory usage
def import_data(file):
    
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df



print('-' * 80)
print('train')

train = import_data('/kaggle/input/ny-ems-incident-dispatch-data/ems-incident-dispatch-data.csv')




--------------------------------------------------------------------------------
train


/tmp/ipykernel_30/2281785750.py:51: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
/tmp/ipykernel_30/2281785750.py:51: DtypeWarning: Columns (7,11,17,21,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)


Memory usage of dataframe is 4908.87 MB


In [ ]:
df = train

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['incident_year'] = pd.to_datetime(df['INCIDENT_DATETIME']).dt.year
df['incident_month'] = pd.to_datetime(df['INCIDENT_DATETIME']).dt.month
df['day_of_week'] = pd.to_datetime(df['INCIDENT_DATETIME']).dt.dayofweek
df['incident_hour'] = pd.to_datetime(df['INCIDENT_DATETIME']).dt.hour

df['INCIDENT_DATETIME'] = pd.to_datetime(df['INCIDENT_DATETIME'])
df['INCIDENT_CLOSE_DATETIME'] = pd.to_datetime(df['INCIDENT_CLOSE_DATETIME'])
df['incident_duration'] = (df['INCIDENT_CLOSE_DATETIME'] - df['INCIDENT_DATETIME']).dt.total_seconds()
df['HELD_INDICATOR'] = df['HELD_INDICATOR'].apply(lambda x: 1 if x == 'Y' else 0)

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
#df = df.ffill(downcast=None, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
encoder = LabelEncoder()
df['INITIAL_CALL_TYPE'] = encoder.fit_transform(df['INITIAL_CALL_TYPE'])
df['FINAL_CALL_TYPE'] = encoder.fit_transform(df['FINAL_CALL_TYPE'])
df['BOROUGH'] = encoder.fit_transform(df['BOROUGH'])

In [ ]:
X = df[['INITIAL_CALL_TYPE', 'INITIAL_SEVERITY_LEVEL_CODE', 'incident_month', 'day_of_week', 'incident_hour', 'BOROUGH']]
y = df['FINAL_CALL_TYPE']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))